In [1]:
# import os
# os.environ["JAX_PLATFORM_NAME"] = "cpu"

import jax_sbgeom as jsb
%load_ext autoreload
%autoreload 2
import jax.numpy as jnp
import jax
jax.config.update("jax_enable_x64", True)
import numpy as onp
import sys 
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

import StellBlanket.SBGeom as SBGeom
from functools import partial
import jax_sbgeom.coils as jsc

from tests.coils.test_coils import _check_single_vectorized
import pyvista as pv
from dataclasses import dataclass

from jax_sbgeom.jax_utils.utils import _mesh_to_pyvista_mesh


vmec_files = ["/home/tbogaarts/stellarator_paper/base_data/vmecs/helias3_vmec.nc4", "/home/tbogaarts/stellarator_paper/base_data/vmecs/helias5_vmec.nc4", "/home/tbogaarts/stellarator_paper/base_data/vmecs/squid_vmec.nc4"]
vmec_file = vmec_files[0]

coil_files = ["/home/tbogaarts/stellarator_paper/base_data/vmecs/HELIAS3_coils_all.h5", "/home/tbogaarts/stellarator_paper/base_data/vmecs/HELIAS5_coils_all.h5", "/home/tbogaarts/stellarator_paper/base_data/vmecs/squid_coilset.h5"]

Triangle Elements not compiled


In [2]:
class bb:
    param : str
    def __init__(self, param):
        self.param = param

def _get_all_discrete_coils(request):    
    coilset_sbgeom = SBGeom.Coils.Discrete_Coil_Set_From_HDF5(request.param)    
    coilset_jaxsbgeom = [jsb.coils.DiscreteCoil.from_positions(coilset_sbgeom[i].Get_Vertices()) for i in range(coilset_sbgeom.Number_of_Coils())]
    
    return coilset_jaxsbgeom, coilset_sbgeom

def _get_all_fourier_coils_trunc(request):
    coilset_sbgeom = SBGeom.Coils.Discrete_Coil_Set_From_HDF5(request.param)
    coilset_fourier = SBGeom.Coils.Convert_to_Fourier_Coils(coilset_sbgeom, Nftrunc = 20)
    
    coilset_jax = [jsb.coils.FourierCoil(jnp.array(i.Get_Fourier_Cos()), jnp.array(i.Get_Fourier_Sin()), jnp.array(i.Get_Centre())) for i in coilset_fourier]
    return coilset_jax, coilset_fourier
    


dcoils_jax, coils_sbgeom = _get_all_discrete_coils(bb(coil_files[0]))
coils_jax, coils_sbgeom = _get_all_fourier_coils_trunc(bb(coil_files[0]))

In [23]:
fs_coil = jsc.FiniteSizeCoil.from_coil(dcoils_jax[0], jsc.CentroidFrame, number_of_rmf_samples = 100)

In [24]:
fs_coil.tangent(0.1)

Array([-0.36266826, -0.25348501,  0.89678151], dtype=float64)

In [26]:
lines = pv.Spline(fs_coil.position(jnp.linspace(0.0, 1.0, 100)))

In [27]:
coil_mesh = jsc.mesh_coil_surface(fs_coil, 2000, 0.5, 0.5)

In [29]:
ss = jnp.linspace(0.0, 1.0, 100, endpoint=False)
rv =   fs_coil.radial_vector(ss)
ps = fs_coil.position(ss)


In [30]:
plotter = pv.Plotter()

plotter.add_mesh(pv.Spline(ps))
plotter.add_arrows(ps, rv , mag=1.0, color='red')
plotter.add_mesh(_mesh_to_pyvista_mesh(coil_mesh[0], coil_mesh[1]), color='lightblue', opacity=1.0)

plotter.show()

Widget(value='<iframe src="http://localhost:45735/index.html?ui=P_0x74ca51829eb0_4&reconnect=auto" class="pyvi…